## Importing libraries and directory settings

In [1]:
import numpy as np
import random
from collections import Counter 

data_dir = '/home/deependra/project/23-hetero-smote/HeteroG/data/imdb_han/'
metadata_file = "movie_metadata.csv"

In [2]:
a = [1,2,3,4,5,6,7,8,9,10]

ValueError: Sample larger than population or is negative

## Loading imdb movie_metadata.csv file

In [2]:
# Loading metadata
with open(data_dir + metadata_file, 'r') as f:
    metadata = f.readlines()
    
lines = [line.strip().split(',') for line in metadata]

print(f"Number of movies: {len(lines)-1}")
print(f"Number of attributes: {len(lines[0])}")

Number of movies: 5043
Number of attributes: 28


In [3]:
# checking if all nodes have same number of features

first_node_features = len(lines[0])

short_indices = []
for i in range(len(lines)):
    if len(lines[i]) != first_node_features:
        short_indices.append(i)
        
print(f"usual number of feautures: {first_node_features}")
print("Number of nodes with less features: ", len(short_indices))
print("Indices of nodes with less features: ", short_indices)
print(f"number of feautures of nodes with less features: {[len(lines[i]) for i in short_indices]}")

print('before: ',lines[65][11:12+(len(lines[65])-first_node_features)])

# correcting the nodes with less features
for i in short_indices:
    temp = ','.join(lines[i][11:12+(len(lines[i])-first_node_features)]).strip()[1:-2]
    lines[i][11:12+(len(lines[i])-first_node_features)] = [temp]
    
print('after; ',lines[65][11])

usual number of feautures: 28
Number of nodes with less features:  69
Indices of nodes with less features:  [65, 239, 275, 280, 689, 817, 898, 924, 954, 1100, 1216, 1502, 1552, 1776, 1785, 1799, 1847, 1890, 1957, 2035, 2108, 2323, 2371, 2437, 2505, 2606, 2795, 2833, 2842, 2877, 3008, 3154, 3183, 3189, 3218, 3233, 3256, 3268, 3325, 3475, 3478, 3552, 3643, 3712, 3731, 3790, 3794, 3944, 4018, 4180, 4221, 4286, 4324, 4384, 4409, 4459, 4485, 4499, 4503, 4555, 4641, 4654, 4840, 4894, 4895, 4964, 4973, 4975, 5003]
number of feautures of nodes with less features: [29, 29, 29, 29, 29, 29, 29, 29, 30, 29, 29, 29, 29, 31, 29, 29, 29, 29, 29, 29, 29, 29, 30, 29, 29, 29, 29, 29, 29, 29, 29, 29, 30, 29, 29, 29, 30, 31, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 30, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 30]
before:  ['"The Chronicles of Narnia: The Lion', ' the Witch and the Wardrobe\xa0"']
after;  The Chronicles of Narnia: The Lion, the Witch and the Wardrobe


In [4]:
print([line[11] for line in lines[:10]])

# stripping movie titles of '\xa0'
for i in range(len(lines)):
    lines[i][11] = lines[i][11].strip()
    
print([line[11] for line in lines[:10]])

['movie_title', 'Avatar\xa0', "Pirates of the Caribbean: At World's End\xa0", 'Spectre\xa0', 'The Dark Knight Rises\xa0', 'Star Wars: Episode VII - The Force Awakens\xa0            ', 'John Carter\xa0', 'Spider-Man 3\xa0', 'Tangled\xa0', 'Avengers: Age of Ultron\xa0']
['movie_title', 'Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'Star Wars: Episode VII - The Force Awakens', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron']


## creating a dictionary of ids of nodes, classes of movies

In [5]:
if lines[0][1] == 'director_name': 
    header = dict(enumerate(lines[0]))
    lines = np.array(lines[1:]) # removing header
print (header)

# removing movies with no directors or actors or plot keywords or titles
remove_idx = []
for i in range(len(lines)):
    if lines[i][1] == '' or lines[i][10] == '' or lines[i][16] == '' or lines[i][11] == '':
        remove_idx.append(i)
print(f"current number of movies: {len(lines)}, removing {len(remove_idx)} movies")
lines = np.delete(lines, remove_idx, axis=0)
print(f"new number of movies: {len(lines)}")

# removing movies with same titles

multipletitle_counts = [i for i in Counter(lines[:,11]).most_common(122) if i[1] > 1]
print(f"Number of movie titles with multiple occurences: {len(multipletitle_counts)}")
title_same_idx = []
remove_idx = []
for title, counts in multipletitle_counts:
    title_same_idx.append(list(np.where(lines[:,11] == title)[0]))
lines2 = {}
for idxs in title_same_idx:
    remove_idx.extend(idxs[1:])
    for idx in idxs:
        lines2[idx] = np.copy(lines[idx])
    
print(f"Number of movies to be removed: {len(remove_idx)}")
lines = np.delete(lines, remove_idx, axis=0)

# checking number of action movies
action_idx = []
for i in range(len(lines)):
    if 'Action' in lines[i][9]:
        action_idx.append(i)
print(f"Number of action movies: {len(action_idx)}")

# checking number of comedy movies
comedy_idx = []
for i in range(len(lines)):
    if 'Comedy' in lines[i][9]:
        comedy_idx.append(i)
print(f"Number of comedy movies: {len(comedy_idx)}")

# checking number of drama movies

drama_idx = []
for i in range(len(lines)):
    if 'Drama' in lines[i][9]:
        drama_idx.append(i)
print(f"Number of drama movies: {len(drama_idx)}")

action_idx = np.array(action_idx)
comedy_idx = np.array(comedy_idx)
drama_idx = np.array(drama_idx)



action_drama_idx = set(action_idx).intersection(set(drama_idx))
print(f"Number of action-drama movies: {len(action_drama_idx)}")

action_comedy_idx = set(action_idx).intersection(set(comedy_idx))
print(f"Number of action-comedy movies: {len(action_comedy_idx)}")

drama_comedy_idx = set(drama_idx).intersection(set(comedy_idx))
print(f"Number of drama-comedy movies: {len(drama_comedy_idx)}")

onlyaction_idx = set(action_idx) - set(action_drama_idx) - set(action_comedy_idx)
print(f"Number of action-only movies: {len(onlyaction_idx)}")

onlydrama_idx = set(drama_idx) - set(action_drama_idx) - set(drama_comedy_idx)
print(f"Number of drama-only movies: {len(onlydrama_idx)}")

onlycomedy_idx = set(comedy_idx) - set(action_comedy_idx) - set(drama_comedy_idx)
print(f"Number of comedy-only movies: {len(onlycomedy_idx)}")

{0: 'color', 1: 'director_name', 2: 'num_critic_for_reviews', 3: 'duration', 4: 'director_facebook_likes', 5: 'actor_3_facebook_likes', 6: 'actor_2_name', 7: 'actor_1_facebook_likes', 8: 'gross', 9: 'genres', 10: 'actor_1_name', 11: 'movie_title', 12: 'num_voted_users', 13: 'cast_total_facebook_likes', 14: 'actor_3_name', 15: 'facenumber_in_poster', 16: 'plot_keywords', 17: 'movie_imdb_link', 18: 'num_user_for_reviews', 19: 'language', 20: 'country', 21: 'content_rating', 22: 'budget', 23: 'title_year', 24: 'actor_2_facebook_likes', 25: 'imdb_score', 26: 'aspect_ratio', 27: 'movie_facebook_likes'}
current number of movies: 5043, removing 255 movies
new number of movies: 4788
Number of movie titles with multiple occurences: 116
Number of movies to be removed: 122
Number of action movies: 1078
Number of comedy movies: 1779
Number of drama movies: 2395
Number of action-drama movies: 313
Number of action-comedy movies: 246
Number of drama-comedy movies: 633
Number of action-only movies: 54

In [6]:
# for experimental purposes, ignore this

special_idx = []
for title_same_check_idx in range(len(title_same_idx)):
    idxs = title_same_idx[title_same_check_idx]
    # print(f"movie1: {lines[idxs[0]][11]}\n movie2: {lines[idxs[1]][11]}")
    # print(f"movie director1: {lines[idxs[0]][1]}\n movie director2: {lines[idxs[1]][1]}")
    # print(f"movie1 actors: {lines[idxs[0]][10],lines[idxs[0]][6],lines[idxs[0]][14]}\n movie2 actors: {lines[idxs[1]][10],lines[idxs[1]][6],lines[idxs[1]][14]}")
    # print(f"movie1 plot keywords: {lines[idxs[0]][16]}\n movie2 plot keywords: {lines[idxs[1]][16]}")
    # print(f"movie1 genres: {lines[idxs[0]][9]}\n movie2 genres: {lines[idxs[1]][9]}")

    diff_idx = [int(i) for i in np.where(lines2[idxs[0]] == lines2[idxs[1]],['' for i in range(len(header))],list(header.keys())) if i != '']
    
    if set(diff_idx).intersection(set([1,11,10,6,14,16,9])):
        special_idx.append(idxs)
    # print(f"Number of different attributes: {len(diff_idx)}")
    # print(f"Different attributes: {[header[i] for i in diff_idx]}")
    # values = [lines2[idxs[0]][i] for i in diff_idx]
    # print(f"Values1: {values}")
    # values = [lines2[idxs[1]][i] for i in diff_idx]
    # print(f"Values2: {values}")
    
    # if len(idxs) > 2:
    #     diff_idx = [i for i in np.where(lines2[idxs[0]] == lines2[idxs[2]],['' for i in range(len(header))],list(header.keys())) if i != '']
    #     if set(diff_idx).intersection(set([1,11,10,6,14,16,9])):
    #         special_idx.append(idxs)
    #     values = [lines2[idxs[2]][i] for i in diff_idx]
    #     print(f"values3: {values}")
    
print(f"{special_idx}")

print(f"Number of very different: {len(special_idx)}")
for idxs in special_idx:
    for j in [1,11,10,6,14,16,9]:
        print(f"{header[j]}: {lines2[idxs[0]][j]}",end=' ')
    print()
    for j in [1,11,10,6,14,16,9]:
        print(f"{header[j]}: {lines2[idxs[1]][j]}",end=' ')
    print()
    if len(idxs) > 2:
        for j in [1,11,10,6,14,16,9]:
            print(f"{header[j]}: {lines2[idxs[2]][j]}",end=' ')
        print()
    print()

[[979, 2910], [1808, 2806], [3677, 3718]]
Number of very different: 3
director_name: Andrew Niccol movie_title: The Host actor_1_name: J.D. Evermore actor_2_name: Chandler Canterbury actor_3_name: Rachel Roberts plot_keywords: alien|alien creature|body snatching|desert|parasite genres: Action|Adventure|Romance|Sci-Fi|Thriller 
director_name: Joon-ho Bong movie_title: The Host actor_1_name: Doona Bae actor_2_name: Kang-ho Song actor_3_name: Ah-sung Ko plot_keywords: daughter|han river|monster|river|seoul genres: Comedy|Drama|Horror|Sci-Fi 

director_name: Jim Sheridan movie_title: Brothers actor_1_name: Natalie Portman actor_2_name: Jake Gyllenhaal actor_3_name: Bailee Madison plot_keywords: afghanistan|death of soldier|presumed dead|prisoner|war trauma genres: Drama|Thriller 
director_name: Jim Sheridan movie_title: Brothers actor_1_name: Natalie Portman actor_2_name: Jake Gyllenhaal actor_3_name: Ethan Suplee plot_keywords: afghanistan|death of soldier|presumed dead|prisoner|war traum

In [7]:
# movie ids same as indices
print(f"Number of movies with no titles: {len([line[11] for line in lines if line[11] == ''])}")
movie_ids = dict(zip(lines[:,11], range(len(lines))))
print(f"Number of movies: {len(movie_ids)}")

# movie class ids
class_ids = {'Action' : 0, 'Comedy' : 1, 'Drama' : 2}

# creating director name id dictionary
print(f"number of movies with no director: {len([line[1] for line in lines if line[1] == ''])}")
director_names = set(lines[:,1]).difference(set(['']))
director_name_id = dict(zip(director_names, range(len(director_names))))
print(f"Number of directors: {len(director_name_id)}")
print(f"Director name id dictionary: {director_name_id}")

# creating actor name id dictionary
print(f"number of movies with no actor 1: {len([line[10] for line in lines if line[10] == ''])}")
print(f"number of movies with no actor 2: {len([line[6] for line in lines if line[6] == ''])}")
print(f"number of movies with no actor 3: {len([line[14] for line in lines if line[14] == ''])}")
print(f"number of movies with no actor 1, 2, 3: {len([line[10] for line in lines if line[10] == '' and line[6] == '' and line[14] == ''])}")
actor_names = set(lines[:,10]).union(set(lines[:,6]),set(lines[:,14])).difference(set(['']))
actor_name_id = dict(zip(actor_names, range(len(actor_names))))
print(f"Number of actors: {len(actor_name_id)}")
print(f"Actor name id dictionary: {actor_name_id}")

Number of movies with no titles: 0
Number of movies: 4666
number of movies with no director: 0
Number of directors: 2271
Director name id dictionary: {'Peyton Reed': 0, 'John Schultz': 1, 'Asghar Farhadi': 2, 'Kasi Lemmons': 3, 'Mike Newell': 4, 'Guillaume Ivernel': 5, 'Carlos Saura': 6, 'Jugal Hansraj': 7, 'Mathieu Kassovitz': 8, 'Reinhard Klooss': 9, 'Jay Levey': 10, 'Jonathan Newman': 11, 'Jonathan Liebesman': 12, "John 'Bud' Cardos": 13, 'Gregory Hoblit': 14, 'François Truffaut': 15, 'Ralph Nelson': 16, 'Hark Tsui': 17, 'Rowdy Herrington': 18, 'Jennifer Yuh Nelson': 19, 'Jim Sheridan': 20, 'George Lucas': 21, 'Gabe Ibáñez': 22, 'Kate Barker-Froyland': 23, 'John Whitesell': 24, 'Gillian Armstrong': 25, 'Jamie Blanks': 26, 'James Fargo': 27, 'Thomas L. Phillips': 28, 'Tom Hooper': 29, 'Barbet Schroeder': 30, 'Edward Norton': 31, 'Bradley Parker': 32, 'Keenen Ivory Wayans': 33, 'Brian Klugman': 34, 'Steve Miner': 35, 'Steve Carver': 36, 'Rob Letterman': 37, 'Dylan Bank': 38, 'Tony Gig

## Creating movie embeddings

In [8]:
plotwords = []
for i in range(len(lines)):
    if lines[i][16] != '':
        plotwords.extend(lines[i][16].split('|'))
print(f"total number of plot words: {len(plotwords)}")

set_plotwords = list(set(plotwords))
print(f"total number of unique plot words: {len(set_plotwords)}")
plotwords_id = dict(zip(set_plotwords, range(len(set_plotwords))))

# counting occurences of each plot word
counts = []
for i in range(len(set_plotwords)):
    counts.append([i, plotwords.count(set_plotwords[i])])
    
counts = np.array(counts)

# sort the counts in descending order using the count values in first column
counts = counts[counts[:,1].argsort()[::-1]]

for i in range(len(counts[:10])):
    print(f"{set_plotwords[counts[i][0]]} : {counts[i][1]}")

total number of plot words: 23011
total number of unique plot words: 7969
love : 189
friend : 159
murder : 153
death : 125
police : 117
new york city : 89
high school : 85
alien : 77
boy : 71
school : 69


In [9]:
# creating movie embeddings 

movie_plotwords = [','.join(line[16].split('|')) for line in lines]

# Import necessary libraries
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize  # You may need to install NLTK

# Sample list of movie keywords (replace this with your actual data)

# Tokenize the keywords
tokenized_keywords = [word_tokenize(keywords) for keywords in movie_plotwords]

# Train a Word2Vec model
model = Word2Vec(sentences=tokenized_keywords, vector_size=128, window=5, min_count=1, sg=1, negative=5)

# Save the model if needed
model.save(data_dir+"movie_keywords_word2vec.model")

# Get embeddings for each movie
movie_embeddings = []
for keywords in tokenized_keywords:
    # Calculate the average vector for each keyword in a movie
    keyword_vectors = [model.wv[keyword] for keyword in keywords if keyword in model.wv]
    if keyword_vectors:
        average_vector = sum(keyword_vectors) / len(keyword_vectors)
        movie_embeddings.append(average_vector)
    else:
        # Handle cases where none of the keywords are in the vocabulary
        movie_embeddings.append(None)

# Now, movie_embeddings contains the embeddings for each movie's plot keywords.
count = 0
for i in range(len(movie_embeddings)):
    if movie_embeddings[i] is None:
        count += 1
        movie_embeddings[i] = [0]*128
print(f"Number of movies with no embeddings: {count}")

movie_embeddings = np.array(movie_embeddings)

Number of movies with no embeddings: 0


## Random walk with restart (RwR)

In [10]:
# creating dictionary of neighbors
# 'd' for director, 'a' for actor, 'm' for movie e.g. 'd0' is director 0, 'a0' is actor 0, 'm0' is movie 0 in their respective ids

all_neighbors = {f'm{i}' : [] for i in range(len(lines))}
all_neighbors.update({f'd{id}' : [] for id in director_name_id.values()})
all_neighbors.update({f'a{id}' : [] for id in actor_name_id.values()})

for i in range(len(lines)):
    all_neighbors[f'm{i}'].append(f'd{director_name_id[lines[i][1]]}')
    all_neighbors[f'd{director_name_id[lines[i][1]]}'].append(f'm{i}')
    
    all_neighbors[f'm{i}'].append(f'a{actor_name_id[lines[i][10]]}')
    all_neighbors[f'a{actor_name_id[lines[i][10]]}'].append(f'm{i}')
    
    if lines[i][6] != '':  
        all_neighbors[f'm{i}'].append(f'a{actor_name_id[lines[i][6]]}')
        all_neighbors[f'a{actor_name_id[lines[i][6]]}'].append(f'm{i}')
    if lines[i][14] != '': 
        all_neighbors[f'm{i}'].append(f'a{actor_name_id[lines[i][14]]}')
        all_neighbors[f'a{actor_name_id[lines[i][14]]}'].append(f'm{i}')

In [11]:
print(f'no. of movies: {len(movie_ids)}')
print(f'no. of directors: {len(director_name_id)}')
print(f'no. of actors: {len(actor_name_id)}')

no. of movies: 4666
no. of directors: 2271
no. of actors: 5845


In [12]:
# step1: random walk with a restart (HAN paper method)

length = 100
prob_restart = 0.5
max_samples = {'m' : 40, 'd' : 20, 'a' : 40} # should add up to length

random_walks = {}
for node in list(all_neighbors.keys()):
    random_walks[node] = []
    curr_node = node
    neighbors = 0
    neigh_m = 0
    neigh_d = 0
    neigh_a = 0
    while neighbors < length:
        p = random.random()
        if p < prob_restart:
            curr_node = node
        else:
            curr_node = random.choice(all_neighbors[curr_node])
            if curr_node[0] == 'm' and neigh_m < max_samples['m']:
                neigh_m += 1
                random_walks[node].append(curr_node)
                neighbors += 1
            elif curr_node[0] == 'd' and neigh_d < max_samples['d']:
                neigh_d += 1
                random_walks[node].append(curr_node)
                neighbors += 1
            elif curr_node[0] == 'a' and neigh_a < max_samples['a']:
                neigh_a += 1
                random_walks[node].append(curr_node)
                neighbors += 1

print(f"number of random walks: {len(random_walks)}")
print(f"singular random walk, {'m0'}: {random_walks['m0']}")

number of random walks: 12782
singular random walk, m0: ['a87', 'm2198', 'a997', 'a87', 'm1644', 'a87', 'd577', 'm25', 'a4074', 'a3760', 'a4074', 'm3583', 'a2769', 'a4074', 'm0', 'd577', 'a87', 'a3760', 'a87', 'a3760', 'a4074', 'd577', 'm282', 'a2146', 'm3246', 'a3760', 'a4074', 'a4074', 'm4166', 'a87', 'a4074', 'd577', 'm282', 'a87', 'd577', 'a4074', 'a87', 'm1090', 'a1370', 'd577', 'm282', 'a4074', 'a87', 'm1090', 'a4074', 'm0', 'a87', 'a4074', 'm0', 'a87', 'm1644', 'd2072', 'm887', 'a87', 'm0', 'a87', 'm1644', 'a3422', 'm3598', 'd577', 'm25', 'a5713', 'a87', 'm0', 'a87', 'm1644', 'a87', 'm1090', 'a87', 'm1644', 'a3122', 'm1644', 'a4074', 'm723', 'd577', 'm282', 'm1644', 'm441', 'm0', 'd577', 'd577', 'm1172', 'm1644', 'm0', 'm3583', 'm616', 'm3438', 'm723', 'd577', 'm441', 'm723', 'd577', 'm0', 'd577', 'd577', 'd577', 'd577', 'd2072', 'd577', 'd200']


## Grouping and finding top neighbors of each node type for all node

In [1]:
# step2: Grouping different types of neighbors based on frequency(HAN paper method)

top_k = {'m' : 8, 'd' : 4, 'a' : 8} # top k neighbors to be considered for each type of node

top_neighbors = {}
for node in tqdm(list(all_neighbors.keys())):
    
    # initializing top_neighbors categores for that node
    top_neighbors[node] = {'m' : [], 'd' : [], 'a' : []}
    
    # finding neighbors of different types
    neigh_a = []
    neigh_d = []
    neigh_m = []
    for neigh in random_walks[node]:
        if neigh[0] == 'm':
            neigh_m.append(neigh)
        elif neigh[0] == 'd':
            neigh_d.append(neigh)
        elif neigh[0] == 'a':
            neigh_a.append(neigh)
    
    # finding top k neighbors (and their countes)
    top_a = Counter(neigh_a).most_common(top_k['a'])
    top_d = Counter(neigh_d).most_common(top_k['d'])
    top_m = Counter(neigh_m).most_common(top_k['m'])
    
    # adding top k neighbors to top_neighbors in nodes respective category
    top_neighbors[node]['a'].extend([i[0] for i in top_a])
    top_neighbors[node]['d'].extend([i[0] for i in top_d])
    top_neighbors[node]['m'].extend([i[0] for i in top_m])


    # adding random neighbors if less than top k
    if len(top_a) < top_k['a']:
        top_neighbors[node]['a'].extend(random.sample(neigh_a,top_k['a'] - len(top_a)))
    if len(top_d) < top_k['d']:
        top_neighbors[node]['d'].extend(random.sample(neigh_d, top_k['d'] - len(top_d)))
    if len(top_m) < top_k['m']:
       top_neighbors[node]['m'].extend(random.sample(neigh_m, top_k['m'] - len(top_m)))

NameError: name 'tqdm' is not defined

## saving movie embeddings as npy and others as txt files

In [14]:
# creating graph data files for ease

# creating metadata processed npy file
np.save(data_dir + 'metadata_np.npy', lines)


# creating movie id file
with open(data_dir + 'movie_id.txt', 'w') as f:
    for i in range(len(lines)):
        f.write(f"{i}: {lines[i][11]}\n")

# creating director_id file
with open(data_dir + 'director_id.txt', 'w') as f:
    for key, value in director_name_id.items():
        f.write(f"{value}, {key}\n")


# creating actor_id file
with open(data_dir + 'actor_id.txt', 'w') as f:
    for key, value in actor_name_id.items():
        f.write(f"{value}, {key}\n")
        
        
# creating movie class file
with open(data_dir + 'm_class_local.txt', 'w') as f:
    for j in onlyaction_idx:
        f.write(f"{j}, {class_ids['Action']}\n")
    for j in onlycomedy_idx:
        f.write(f"{j}, {class_ids['Comedy']}\n")
    for j in onlydrama_idx:
        f.write(f"{j}, {class_ids['Drama']}\n")


# creating movie director edge index file
with open(data_dir + 'movie_director.txt', 'w') as f:
    for i in range(len(lines)):
        if lines[i][1] != '':
            f.write(f"{i}, {director_name_id[lines[i][1]]}\n")
        
        
# creating movie actor edge index file
with open(data_dir + 'movie_actors.txt', 'w') as f:
    for i in range(len(lines)):
        if lines[i][10] != '':
            f.write(f"{i}, {actor_name_id[lines[i][10]]}\n")
        if lines[i][6] != '':
            f.write(f"{i}, {actor_name_id[lines[i][6]]}\n")
        if lines[i][14] != '':
            f.write(f"{i}, {actor_name_id[lines[i][14]]}\n")
            
            
# creating movie_embedding file
with open(data_dir + 'movie_embeddings.txt', 'w') as f:
    for i in range(len(movie_embeddings)):
        f.write(f"{' '.join([str(j) for j in movie_embeddings[i]])}\n")
        
        
# creating all neighbors file
with open(data_dir + 'node_neighbors_all.txt', 'w') as f:
    for key, value in all_neighbors.items():
        f.write(f"{key}: {','.join(value)}\n")
        
# creating random walks file
with open(data_dir + 'random_walks.txt', 'w') as f:
    for key, value in random_walks.items():
        f.write(f"{key}: {','.join(value)}\n")
        
# creating top neighbors file
with open(data_dir + 'node_neighbors_top.txt', 'w') as f:
    for key, value in top_neighbors.items():
        f.write(f"{key}: {','.join(value['m'])};{','.join(value['d'])};{','.join(value['a'])}\n")